In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Read and save the original data
data = pd.read_excel('../data/data-1.xlsx')
original_data = data.copy()

# Delete features with varience less than 0.1
variances = original_data.iloc[:, 2:].var() 
variance_threshold = 0.1
features_to_keep = variances[variances > variance_threshold].index

# Z-score normalization
scaler = StandardScaler()
feature_columns = data.columns.drop(['PFAS', 'LogKd'])
data[feature_columns] = scaler.fit_transform(data[feature_columns])

# Remove collinear features Using Spearmangit correlation greater than 0.8
correlation_matrix = data.iloc[:, 2:].corr(method='spearman')
threshold = 0.80
columns_to_drop = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > threshold:
            colname = correlation_matrix.columns[i]
            columns_to_drop.append(colname)

data_reduced = data.drop(columns=list(set(columns_to_drop)))

# Select the retained features from the original data
data_final = original_data[['PFAS', 'LogKd'] + list(features_to_keep)]
duplicates = data_final.columns[data_final.columns.duplicated()]
data_final.drop(columns=duplicates, inplace=True)

data_final.to_excel('../data/data-2.xlsx', index=False)

C:\Users\20529\AppData\Local\Temp\ipykernel_9872\2491817778.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_final.drop(columns=duplicates, inplace=True)


In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor

# Read data
data = pd.read_excel('../data/data-2.xlsx')
data.fillna(0, inplace=True)

X = data.drop(columns=['PFAS', 'LogKd'])
y = data['LogKd']

# Caculate mean and std to remove outliers
mean = data.iloc[:, 1].mean()
std = data.iloc[:, 1].std()
data = data[(data.iloc[:, 1] > mean - 3*std) & (data.iloc[:, 1] < mean + 3*std)].dropna()

# Using RF to caculate feature importance, repeat 100 times
feature_importances = np.zeros(X.shape[1])

n_repeats = 100
for _ in range(n_repeats):
    model = RandomForestRegressor()
    model.fit(X, y)
    feature_importances += model.feature_importances_
    feature_importances /= n_repeats

# Get the indices of important features and read the corresponding data to save
top_indices = np.argsort(feature_importances)[-50:]
top_features = X.columns[top_indices]
top_features_data = data[top_features]
top_features_data['LogKd'] = y

top_features_data.to_excel('../data/data-2-1.xlsx', index=False)

C:\Users\20529\AppData\Local\Temp\ipykernel_9872\1741519302.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_features_data['LogKd'] = y
